In [1]:
from dotenv import dotenv_values
from datasets import load_dataset, Dataset
from utils.data_preprocessor import DataPreprocessor
from utils.evaluator import Evaluator
from config.finetuning import config
from utils.load_merged_model_tokenizer import load_mergedModel_tokenizer
from config import postprocessing
from utils.test_data_processor import TestDataProcessor
import pandas as pd
from log import enlayer1_3epochs_4bits__ft_params as models_params
from utils.generate_ft_adapters_list import generate_ft_adapters_list
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gc
from peft import PeftModel
from tqdm import tqdm

HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']

max_new_tokens_factor_list = postprocessing.max_new_tokens_factor_list
n_shots_inference_list = postprocessing.n_shots_inference_list
layer = models_params.TRAIN_LAYER
language = layer.split('.')[0]


dataset = load_dataset("ferrazzipietro/e3c-sentences", token=HF_TOKEN)
dataset = dataset[layer]
preprocessor = DataPreprocessor()
dataset = preprocessor.preprocess_data_one_layer(dataset)
_, val_data, _ = preprocessor.split_layer_into_train_val_test_(dataset, layer)

bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16)


adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")



/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
for max_new_tokens_factor in max_new_tokens_factor_list:
    for n_shots_inference in n_shots_inference_list:
        for adapters in tqdm(adapters_list, desc="adapters_list"):
            print("PROCESSING:", adapters)
            base_model = AutoModelForCausalLM.from_pretrained(
                models_params.BASE_MODEL_CHECKPOINT, low_cpu_mem_usage=True,
                quantization_config = bnb_config,
                return_dict=True,  load_in_4bit=True, 
                #torch_dtype=torch.float16,
                device_map= "auto")
            merged_model = PeftModel.from_pretrained(base_model, adapters, token=HF_TOKEN, device_map='auto')
            tokenizer = AutoTokenizer.from_pretrained(models_params.BASE_MODEL_CHECKPOINT, add_eos_token=True)
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.padding_side = "left"

            # merged_model, tokenizer = load_mergedModel_tokenizer(adapters, base_model)
            postprocessor = TestDataProcessor(test_data=val_data, 
                                              preprocessor=preprocessor, 
                                              n_shots_inference=n_shots_inference, 
                                              language=language, 
                                              tokenizer=tokenizer)
            postprocessor.add_inference_prompt_column()
            postprocessor.add_ground_truth_column()
            try:
                postprocessor.add_responses_column(model=merged_model, 
                                                tokenizer=tokenizer, 
                                                batch_size=24, 
                                                max_new_tokens_factor=max_new_tokens_factor)
                postprocessor.test_data.to_csv(f"data/test_data_processed/maxNewTokensFactor{max_new_tokens_factor}_nShotsInference{n_shots_inference}_{adapters.split('/')[1]}.csv", index=False)
            except Exception as e:
                print("ERROR IN PROCESSING: ", Exception)
            del merged_model
            del base_model
            del tokenizer
            gc.collect()
            torch.cuda.empty_cache()



adapters_list:   0%|          | 0/36 [00:00<?, ?it/s]

PROCESSING: ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002


adapters_list:   3%|▎         | 1/36 [00:50<29:34, 50.69s/it]

PROCESSING: ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.002


adapters_list:   6%|▌         | 2/36 [00:59<14:41, 25.92s/it]

ERROR IN PROCESSING:  <class 'Exception'>
PROCESSING: ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_4_0.0002


adapter_config.json: 100%|██████████| 552/552 [00:00<00:00, 461kB/s]
adapter_model.safetensors: 100%|██████████| 92.3M/92.3M [00:01<00:00, 46.6MB/s]
adapters_list:   6%|▌         | 2/36 [01:35<27:05, 47.81s/it]


KeyboardInterrupt: 

### ONE RUN

In [4]:
from utils.load_merged_model_tokenizer import load_tokenizer

adapters = "ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.002"
merged_model, tokenizer = load_mergedModel_tokenizer(adapters, "mistralai/Mistral-7B-Instruct-v0.2")


from_backup_file = False
if from_backup_file:
    tmp_data = pd.read_csv(f"data/test_data_processed/en_nShots{n_shots_inference}_maxNewTokensFactor{max_new_tokens_factor}.csv")
    tmp_data = Dataset.from_pandas(tmp_data)
    postprocessor = TestDataProcessor(test_data=tmp_data, preprocessor=preprocessor, n_shots_inference=2, language='en', tokenizer=tokenizer)

if not from_backup_file:
    postprocessor = TestDataProcessor(test_data=val_data.select(range(6)), preprocessor=preprocessor, n_shots_inference=2, language='en', tokenizer=tokenizer)
    postprocessor.add_inference_prompt_column()
    postprocessor.add_ground_truth_column()
    postprocessor.add_responses_column(model=merged_model, tokenizer=tokenizer, batch_size=3, max_new_tokens_factor=5)
    postprocessor.test_data.to_csv(f"data/test_data_processed/en_nShots{n_shots_inference}_maxNewTokensFactor{max_new_tokens_factor}.csv", index=False)

generating responses:   0%|          | 0/6 [00:02<?, ?it/s]


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0